In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/implant_classifier/pytorch/default/1/implant_classifier.pth
/kaggle/input/testest4/implant.jpg
/kaggle/input/testest3/doubleImplants.jpg
/kaggle/input/oii-dds-resized-5000/oii-data-resized-implant/single_implants/15020-B-01_1.jpg
/kaggle/input/oii-dds-resized-5000/oii-data-resized-implant/single_implants/16515-B-02_4.jpg
/kaggle/input/oii-dds-resized-5000/oii-data-resized-implant/single_implants/15102-B-01_4.jpg
/kaggle/input/oii-dds-resized-5000/oii-data-resized-implant/single_implants/14448-B-02_2.jpg
/kaggle/input/oii-dds-resized-5000/oii-data-resized-implant/single_implants/15180-B-01_3.jpg
/kaggle/input/oii-dds-resized-5000/oii-data-resized-implant/single_implants/12272-B-02_0.jpg
/kaggle/input/oii-dds-resized-5000/oii-data-resized-implant/single_implants/15544-B-03_1.jpg
/kaggle/input/oii-dds-resized-5000/oii-data-resized-implant/single_implants/16091-B-02_5.jpg
/kaggle/input/oii-dds-resized-5000/oii-data-resized-implant/single_implants/16040-B-01_0.jpg
/kaggle/inpu

In [2]:
import os
from PIL import Image
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(), 
    transforms.Normalize([0.5], [0.5])
])
dataset_path = "/kaggle/input/oii-dds-resized-5000/oii-data-resized-implant"
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.75 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
class_names = full_dataset.classes
print(f"Classes: {class_names}")
print(f"Training size: {len(train_dataset)}, Validation size: {len(val_dataset)}, Testing size: {len(test_dataset)}")

Classes: ['compound', 'double_implants', 'others', 'single_implants', 'steel_ball']
Training size: 18750, Validation size: 3750, Testing size: 2500


In [4]:
model = models.resnet50(pretrained=True)
num_classes = len(class_names)
model.fc = nn.Linear(model.fc.in_features, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(model)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 222MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

Epoch 1/5, Loss: 0.2157316060050203
Epoch 2/5, Loss: 0.08431622191290959
Epoch 3/5, Loss: 0.06536871458939356
Epoch 4/5, Loss: 0.06508546943096691
Epoch 5/5, Loss: 0.039216672177949


In [8]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))

Test Accuracy: 0.9848
Classification Report:
                 precision    recall  f1-score   support

       compound       1.00      1.00      1.00       499
double_implants       1.00      1.00      1.00       480
         others       0.96      0.97      0.96       498
single_implants       0.98      0.97      0.97       507
     steel_ball       0.99      0.99      0.99       516

       accuracy                           0.98      2500
      macro avg       0.98      0.98      0.98      2500
   weighted avg       0.98      0.98      0.98      2500



In [9]:
torch.save(model.state_dict(), "implant_classifier.pth")
print("Model saved as 'implant_classifier.pth'")

Model saved as 'implant_classifier.pth'


In [12]:
test_image_path = "/kaggle/input/testest3/doubleImplants.jpg"
image = Image.open(test_image_path).convert('RGB')
input_tensor = transform(image).unsqueeze(0).to(device)
with torch.no_grad():
    outputs = model(input_tensor)
    _, predicted_class = torch.max(outputs, 1)
predicted_label = class_names[predicted_class.item()]
print(f"Predicted Implant Type: {predicted_label}")

Predicted Implant Type: double_implants


In [8]:
from PIL import Image
from torchvision import transforms
from torchvision import models
import torch.nn as nn
import torch

model = models.resnet50(pretrained=False)
num_classes = 5
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("/kaggle/input/implant_classifier/pytorch/default/1/implant_classifier.pth", map_location=torch.device('cpu')))
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/tmp/ipykernel_24/2976001557.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/input/implant_classifier/pytorch/default/1/implan

In [13]:
from torchvision import datasets, transforms, models

dataset_path = "/kaggle/input/oii-dds-resized-5000/oii-data-resized-implant"
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])
class_names = full_dataset.classes
image_path = "/kaggle/input/test/implant.jpg" 
image = Image.open(image_path).convert('RGB')
input_tensor = transform(image).unsqueeze(0).to(device)
with torch.no_grad():
    outputs = model(input_tensor)
    _, predicted_class = torch.max(outputs, 1)
predicted_label = class_names[predicted_class.item()]
print(f"Predicted Implant Type: {predicted_label}")

Predicted Implant Type: single_implants
